In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 反馈或问题？
有任何反馈或问题，请打开一个[问题](https://github.com/googleapis/python-aiplatform/issues)。

Python的Vertex SDK：自定义表格训练（异步）示例

要使用这个Colaboratory笔记本，您需要将笔记本复制到您自己的谷歌云盘，并在Colaboratory（或Colab）中打开它。您可以运行每一步，或单元格，并查看其结果。要运行一个单元格，使用Shift+Enter。Colab会自动显示每个单元格中最后一行的返回值。有关在Colab中运行笔记本的更多信息，请参阅[Colab欢迎页面](https://colab.research.google.com/notebooks/welcome.ipynb)。

这个笔记本演示了如何基于表格数据集创建一个自定义模型（异步）。这将需要您提供一个存储数据集CSV文件的存储桶。

注意：在测试此SDK时，您可能会因为训练、预测、存储或使用其他GCP产品而产生费用。

# 安装Python的Vertex SDK，进行验证，并将数据集上传到您的GCS桶

安装完SDK后，内核将自动重新启动。您可能会看到这个错误消息`您的会话因未知原因而崩溃`，这是正常的。

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 输入您的项目和GCS存储桶

在下面的单元格中输入您的项目ID。然后运行该单元格以确保Cloud SDK在这个笔记本中的所有命令中使用正确的项目。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
MY_PROJECT = "YOUR PROJECT"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as Vertex AI

我们正在使用的数据集是鲍鱼数据集。有关此数据集的更多信息，请访问：https://archive.ics.uci.edu/ml/datasets/abalone

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv
!gsutil cp abalone_train.csv {MY_STAGING_BUCKET}/data/

gcs_csv_path = f"{MY_STAGING_BUCKET}/data/abalone_train.csv"

初始化 Python 的 Vertex SDK

为 Vertex AI 初始化 *client*

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

从CSV创建一个托管的表格数据集

托管数据集可用于创建AutoML模型或自定义模型。

In [ ]:
ds = aiplatform.TabularDataset.create(
    display_name="abalone", gcs_source=[gcs_csv_path], sync=False
)

# 编写训练脚本
- 将此单元格写入文件中，以供自定义训练使用。

In [ ]:
%%writefile training_script.py

import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# uncomment and bump up replica_count for distributed training
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# tf.distribute.experimental_set_strategy(strategy)

col_names = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Age"]
target = "Age"

def aip_data_to_dataframe(wild_card_path):
    return pd.concat([pd.read_csv(fp.numpy().decode(), names=col_names)
                      for fp in tf.data.Dataset.list_files([wild_card_path])])

def get_features_and_labels(df):
    return df.drop(target, axis=1).values, df[target].values

def data_prep(wild_card_path):
    return get_features_and_labels(aip_data_to_dataframe(wild_card_path))


model = tf.keras.Sequential([layers.Dense(64), layers.Dense(1)])
model.compile(loss='mse', optimizer='adam')

model.fit(*data_prep(os.environ["AIP_TRAINING_DATA_URI"]),
          epochs=10 ,
          validation_data=data_prep(os.environ["AIP_VALIDATION_DATA_URI"]))
print(model.evaluate(*data_prep(os.environ["AIP_TEST_DATA_URI"])))

# save as Vertex AI Managed model
tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

# 启动培训任务来创建模型

一旦我们定义好您的训练脚本，我们将创建一个模型。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="train-abalone-dist-1-replica",
    script_path="training_script.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest",
    requirements=["gcsfs==0.7.1"],
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)
model = job.run(ds, replica_count=1, model_display_name="abalone-model", sync=False)

部署您的模型，然后等待模型完成部署后再进行预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4", sync=False)

等待部署完成。

In [ ]:
endpoint.wait()

# 在端点上进行预测

In [ ]:
prediction = endpoint.predict(
    [
        [0.435, 0.335, 0.11, 0.33399999999999996, 0.1355, 0.0775, 0.0965],
        [0.585, 0.45, 0.125, 0.874, 0.3545, 0.2075, 0.225],
    ]
)
prediction